In [1]:
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

['preserve_unused_tokens=False']

In [2]:
pip install bert-tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Downloading tokenization script created by the Google
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

from bert import tokenization

In [5]:
img_feature = np.load('Emb_feature/new_test_features/.npy')
np.shape(img_feature)

(8313, 100, 1024)

In [6]:
FMS = pd.read_csv("FMS_final.csv")
FMS['textNdesc'] = 'In the picture '+ FMS.gen_caption + ' And the text says: ' + FMS.text
print(FMS.textNdesc[0], '\n\n')
print(len(FMS))

In the picture man in black and white cap is holding up sign . And the text says: its their character not their color that matters 


8313


In [7]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [8]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    image_input = tf.keras.Input(shape=(100,1024), batch_size=None, name="image_input")
    image_flatten = tf.keras.layers.Flatten()(image_input)
    image_dense_1 = tf.keras.layers.Dense(1024, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=54))(image_flatten)
    image_dense_2 = tf.keras.layers.Dense(1024, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=32))(image_dense_1)


    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

    clf_output = sequence_output[:, 0, :]

    print(np.shape(sequence_output))
    print(np.shape(clf_output))

    image_question = tf.keras.layers.Multiply()([image_dense_2, clf_output])
    image_question_dense_1 = tf.keras.layers.Dense(1000, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=19))(image_question)
    image_question_dense_2 = tf.keras.layers.Dense(1000, activation = tf.nn.relu, kernel_initializer = tf.keras.initializers.he_uniform(seed=28))(image_question_dense_1)
    output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, kernel_initializer = tf.keras.initializers.glorot_normal(seed=15))(image_question_dense_2)
    
    # out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids, image_input], outputs=output)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
    return model

In [9]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

Wall time: 18.9 s


In [10]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [11]:
train_input = bert_encode(FMS.textNdesc.values, tokenizer, max_len=160)
train_labels = FMS.label.values

In [14]:
np.shape(train_input)

(3, 8313, 160)

In [12]:
model = build_model(bert_layer, max_len=160)
model.summary()

(None, 160, 1024)
(None, 1024)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 100, 1024)]  0           []                               
                                                                                                  
 flatten (Flatten)              (None, 102400)       0           ['image_input[0][0]']            
                                                                                                  
 input_word_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 160)]        0           []                               
                                                               

c:\Users\yasir\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
%%time
checkpoint = ModelCheckpoint('model4.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    [train_input, img_feature], train_labels,
    validation_split=0.3,
    epochs=100 ,
    callbacks=[checkpoint],
    batch_size=32,
    steps_per_epoch=12   
)

Epoch 1/100
12/12 [==============================] - 1453s 129s/step - loss: 0.9476 - accuracy: 0.6276 - auc: 0.4819 - val_loss: 0.6976 - val_accuracy: 0.5938 - val_auc: 0.5521
Epoch 2/100
12/12 [==============================] - 1391s 125s/step - loss: 0.6844 - accuracy: 0.6068 - auc: 0.5422 - val_loss: 0.6968 - val_accuracy: 0.5674 - val_auc: 0.6113
Epoch 3/100
12/12 [==============================] - 1432s 129s/step - loss: 0.6747 - accuracy: 0.6354 - auc: 0.6000 - val_loss: 0.6892 - val_accuracy: 0.5958 - val_auc: 0.6598
Epoch 4/100
12/12 [==============================] - 1418s 127s/step - loss: 0.6702 - accuracy: 0.6276 - auc: 0.6223 - val_loss: 0.6809 - val_accuracy: 0.5966 - val_auc: 0.6684
Epoch 5/100
12/12 [==============================] - 1383s 124s/step - loss: 0.6504 - accuracy: 0.6693 - auc: 0.5705 - val_loss: 0.6292 - val_accuracy: 0.6544 - val_auc: 0.6759
Epoch 6/100
12/12 [==============================] - 1368s 123s/step - loss: 0.6669 - accuracy: 0.6458 - auc: 0.629